# Tutorial 2: scRNA and ST data integration (deconvolution)

In this tutorial, we show how to apply DeepST to integrate scRNA-seq and ST data, i.e., deconvolution. Taking human lymph node dataset as example, both scRNA-seq and ST data were downloaded from an existing study by Kleshchevnikov et al. and provided at https://drive.google.com/drive/folders/1ns-EsWBu-SNrJ39j-q-AFIV5U-aXFwXf. 

After downloading the data, we can obtain 'scRNA.h5ad' and 'ST.h5ad' files, which are corresponding reference and spatial transcriptomics data respectively. Cell type information is included in scRNA.obs['cell_type'].  

In [1]:
import scanpy as sc
from DeepST import DeepST

In [2]:
dataset = 'Human_Lymph_Node'

In [3]:
# set random seed
random_seed = 50 
DeepST.fix_seed(random_seed)

In [4]:
# read ST data
file_fold = '/home/yahui/Yahui/Projects/data/Human_Lymph_Node/' #Please replace it with the ST download path 
adata = sc.read_h5ad(file_fold + 'ST.h5ad')
    
#For '10X' ST data, please read it instead as:
#file_fold = '/home/yahui/Yahui/Projects/data/' + str(dataset) #Please replace it with the ST download path 
#adata = sc.read_visium(file_fold, count_file='filtered_feature_bc_matrix.h5', load_images=True)

adata.var_names_make_unique()

In [5]:
# preprocessing for ST data
DeepST.preprocess(adata)

# build graph
DeepST.construct_interaction(adata)
DeepST.add_contrastive_label(adata)

In [6]:
# read scRNA daa
file_path = '/home/yahui/Yahui/Projects/data/Human_Lymph_Node/scRNA.h5ad' #Please replace it with the scRNA download path. 
adata_sc = sc.read(file_path)
adata_sc.var_names_make_unique()

In [7]:
# preprocessing for scRNA data
DeepST.preprocess(adata_sc)

In [8]:
# find overlap genes
from DeepST.preprocess import filter_with_overlap_gene
adata, adata_sc = filter_with_overlap_gene(adata, adata_sc)

Number of overlap genes: 1313


/home/yahui/anaconda3/envs/long/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


In [9]:
# get features
DeepST.get_feature(adata)

#print('adata:', adata)
#print('adata_sc:', adata_sc)

In [ ]:
# Train model
model = DeepST.Train(adata, adata_sc, epochs=1200, deconvolution=True)
adata, adata_sc = model.train_map()

Begin to train ST data...


100%|███████████████████████████████████████| 1200/1200 [00:14<00:00, 81.48it/s]


Optimization finished for ST data!
Begin to train scRNA data...


100%|███████████████████████████████████████| 1200/1200 [00:20<00:00, 59.75it/s]


Optimization finished for cell representation learning!
Begin to learn mapping matrix...


 96%|█████████████████████████████████████▏ | 1146/1200 [01:56<00:05,  9.72it/s]

After model training, we can obtain the learned mapping matrix with dimension 'n_spot x n_cell' in adata.obsm['map_matrix']. 
Each element in the mapping matrix denotes the mapping probability of a cell in a given spot. To filter out noise, we only consider
the top 'retain_percent' cell values for each spot.

We usually set the 'retain_percent' value as 0.15. Users can change the parameter according to your requirement. 

In [ ]:
# Project cells into spatial space
from DeepST.utils import project_cell_to_spot
project_cell_to_spot(adata, adata_sc, retain_percent=0.15)

After projection, the probability distributions of each cell type in spots are saved in adata.obs.

In [ ]:
adata

In [ ]:
# Visualization of spatial distribution of scRNA-seq data
import matplotlib as mpl
with mpl.rc_context({'axes.facecolor':  'black',
                     'figure.figsize': [4.5, 5]}):

         sc.pl.spatial(adata, cmap='magma',
                  # selected cell types
                  color=['B_Cycling', 'B_GC_LZ', 'B_GC_DZ', 'B_GC_prePB'],
                  ncols=4, size=1.3,
                  img_key='hires',
                  # limit color scale at 99.2% quantile of cell abundance
                  vmin=0, vmax='p99.2'
                 )